<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400&display=swap" rel="stylesheet">
<div style="font-family: 'Inter'; font-size: 34px; color: #749857;"><B>🍏 Show Up</b> for Health</div>
<div style="font-family: 'Inter'; font-size: 14px; color: #7a7979;"><B>Predicting Missed Apppointment in Primary Care</b> - Brompton Health PCN</div>

[janduplessis883](https://github.com/janduplessis883)<BR>
[AlexAlexRose](https://github.com/AlexAlexRose)<BR>
[FabySp](https://github.com/FabySp)<BR>
[mogleyza](https://github.com/mogleyza)

# janduplessis883 - Feature Engineering

### Importing Libraries

In [1]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

### Params

In [2]:
RAW_DATA = '~/code/janduplessis883/data-showup/data/raw-data/'
WEATHER_DATA = '~/code/janduplessis883/data-showup/data/weather/weather.csv'
IMD_DATA = '~/code/janduplessis883/data-showup/data/imd-master/imd_master.csv'

OUTPUT_DATA = '~/code/janduplessis883/data-showup/data/output-data/'

In [7]:
data = pd.read_csv(f'{OUTPUT_DATA}training_data_no_features.csv')
data.head()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
1,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
2,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
3,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589,2022-07-05 09:00:00,17.7,0.0,40,W8 6JQ,Female,2014-01-07 00:00:00,Other White,(XaG62) Main spoken language Portuguese,Current,0.00,0,0,0,0,0,0,0,51.496001,-0.193468,22698.0,0.519419,0.816996
4,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135,2022-07-05 10:00:00,18.5,0.0,70,SW3 3SQ,Male,2017-03-06 00:00:00,Indian or British Indian,(XaG5t) Main spoken language English,Current,0.39,0,1,0,1,1,0,0,51.490771,-0.168178,10169.0,0.540792,1.656309


In [8]:
data.shape

(1026757, 31)

In [6]:
data.to_csv('training_data_no_features.csv', index=False)

In [9]:
data['Registration status'].value_counts()

Current               908518
Deducted               85123
Deceased, Deducted     32986
Deceased                 130
Name: Registration status, dtype: int64

### Only Keep Current Patients

In [10]:
mask = data['Registration status'] == 'Current'

In [11]:
filtered_data = data[mask]

In [12]:
filtered_data.shape

(908518, 31)

In [13]:
filtered_data.duplicated().sum()

0

### Ethnicity Category

In [18]:
filtered_data.groupby(['Sex', 'Appointment status']).count()


Appointment booked date  Appointment date  \
Sex           Appointment status                                              
Female        Arrived                                1530              1530   
              Did Not Attend                        22088             22088   
              Finished                             550653            550653   
              In Progress                             769               769   
              Patient Walked Out                      307               307   
              Waiting                                  58                58   
Indeterminate Did Not Attend                            3                 3   
              Finished                                 49                49   
Male          Arrived                                 937               937   
              Did Not Attend                        12831             12831   
              Finished                             318623            318623   
              In Progress                             439               439   
              Patient Walked Out                      176               176   
              Waiting                                  33                33   
Unknown       Finished                                 22                22   

                                  Appointment time  Booked by  Clinician  \
Sex           Appointment status                                           
Female        Arrived                         1530       1530       1530   
              Did Not Attend                 22088      22088      22088   
              Finished                      550653     550653     550653   
              In Progress                      769        769        769   
              Patient Walked Out               307        307        307   
              Waiting                           58         58         58   
Indeterminate Did Not Attend                     3          3          3   
              Finished                          49         49         49   
Male          Arrived                          937        937        937   
              Did Not Attend                 12831      12831      12831   
              Finished                      318623     318623     318623   
              In Progress                      439        439        439   
              Patient Walked Out               176        176        176   
              Waiting                           33         33         33   
Unknown       Finished                          22         22         22   

                                  Rota type  Patient ID  app datetime    temp  \
Sex           Appointment status                                                
Female        Arrived                  1530        1530          1530    1530   
              Did Not Attend          22088       22088         22088   22088   
              Finished               550653      550653        550653  550653   
              In Progress               769         769           769     769   
              Patient Walked Out        307         307           307     307   
              Waiting                    58          58            58      58   
Indeterminate Did Not Attend              3           3             3       3   
              Finished                   49          49            49      49   
Male          Arrived                   937         937           937     937   
              Did Not Attend          12831       12831         12831   12831   
              Finished               318623      318623        318623  318623   
              In Progress               439         439           439     439   
              Patient Walked Out        176         176           176     176   
              Waiting                    33          33            33      33   
Unknown       Finished                   22          22            22      22   

                   